# Understanding Recurrence (how to infer long-term rates)

One of the most critical components of any seismic hazard analysis is an estimate of the long-term rates of seismicity from your region or seismogenic source. 

So ... the questions we will try to answer are:<b>

1. For a given region how often could we expect to see earthquakes larger than a certain magnitude?

2. What is the probability of observing this earthquake in a 50 year period?</b>

Arguably the most well-established observations of seismology is the <b>Gutenberg & Richter (1944)</b> model which describes the cumulative rate of events above a given magnitude ($N_C$) to the magnitude itself via:

$\log_{10} N_C = a - b \cdot m$

Theoretically this simple linear relation is easy to fit to earthquake data, but there is a complication! As we saw in the completeness notebook, it is common to have unequal periods of observational completeness across the entire magnitude range. If we don't correct for this somehow then our estimates will be biased.

This notebook will show you how to do this using the magnitude and time data in the completeness notebook. We will consider here two methods (both of which have precedent for use in real applications):

1. Weighted Least Squares

2. Weichert (1980) Maximum Likelihood

Let's get started ...

In [ ]:
%matplotlib inline
# Out usual numerical and plotting libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# A function to find the root of an equation - we'll come to this later!
from scipy.optimize import fsolve

# Copied across from the Completeness notebook!
def plot_magnitude_time_density(magnitudes, years, mbins, time_bins,
                                completeness_table=[], vmin=1, vmax=100,
                                filename=None, filetype="png", dpi=300):
    """
    Create a magnitude density plot
    :param magnitudes:
        Vector of magnitudes
    :param years:
        Vector of years
    :param mbins:
        Edges of the magnitude bins
    :param time_bins:
        Edges of the time bins
    :param completeness_table:
        If present, the table of completeness
    """
    # Generate a 2-D historgram in terms of magnitude and time
    counter = np.histogram2d(years, magnitudes, bins=(time_bins,mbins))[0]
    # Plot the density
    plt.figure(figsize=(10,6))
    plt.pcolormesh(time_bins[:-1], mbins[:-1],
                   counter.T, norm=LogNorm(vmin, vmax))
    # Add axes and labels
    plt.xlabel("Year", fontsize=16)
    plt.ylabel("Magnitude", fontsize=16)
    plt.tick_params(labelsize=14)
    plt.colorbar()
    plt.grid()
    # If a completeness table is given add on a step line
    if len(completeness_table):
        completeness = np.array(completeness_table)
        plt.step(completeness[:, 0], completeness[:, 1], "k--", lw=2)
    # If the filename is specified then export to file 
    if filename:
        plt.savefig(filename, format=filetype, dpi=300, bbox_inches="tight")

Load in the catalogue we used previously in the completeness notebook

In [ ]:
# Load in the catalogue from the completeness notebook
catalogue = np.genfromtxt("./test_completeness_catalogue_1.csv", delimiter=",", skip_header=1)
# Sort into it's magnitudes and years
magnitudes = catalogue[:, 0]
years = catalogue[:, 1]

In [ ]:
# Take the same completeness table you found in the completeness notebook ...
completeness_table = [[2005, 3.0],
                      [2005, 3.5],
                      [1990, 4.0],
                      [1975, 4.5],
                      [1960, 5.0],
                      [1950, 5.5],
                      [1900, 6.0],
                      [1900, 6.5]]
# ... and turn it into a numpy array
completeness_table = np.array(completeness_table)

In [ ]:
# As we did before, let's take a look at the density of events
magnitude_bins = np.arange(3.0, 7.1, 0.1)
year_bins = np.arange(1900., 2016., 1)
plot_magnitude_time_density(magnitudes, years,
                            mbins=magnitude_bins,
                            time_bins=year_bins,
                            completeness_table=completeness_table)

### Now to the main work

The core principal behind the methods we will look at are that we need to find the best estimate of the rate of events in different magnitude bins. <i>However</i> we can only make this estimation by adjusting for the period of time for which we believe that bin is <i>complete</i>.

In this example, we use our completeness table to define the bins for magnitude, which gives the following as the bins:

`3.0 <= M < 3.5`

`3.5 <= M < 4.0`

`4.0 <= M < 4.5`

`4.5 <= M < 5.0`

`5.0 <= M < 5.5`

`5.5 <= M < 6.0`

`6.0 <= M < 6.5`

`6.5 <= M < 7.0`

So, we need to count in each bin the number of earthquakes within the completeness period and the duration of the completeness period

Pay attention step-by-step here ...

In [ ]:
# Our catalogue ends at the end of 2015
end_year = 2015.
# Number of bins is taken from the number of rows in the completeness table
nbins = completeness_table.shape[0]
# The width of the bins is 0.5 M
bin_width = 0.5
# We will count the number of earthquakes, rates, midpoints of the bins and duration - so start with zeros
n_mags = np.zeros(nbins)
rates = np.zeros_like(n_mags)
midpoints = np.zeros_like(n_mags)
duration = np.zeros_like(n_mags)
# Loop through each row in the completeness table
for i, row in enumerate(completeness_table):
    # Starting year is the completeness year
    start_year = row[0]
    # Duration is the time between the end of the catalogue (incl. 2015) and
    # and the year of completeness 
    duration[i] = end_year - start_year + 1
    # A bit of logical work entering here, select the years >= the completeness year
    selected_years = years >= start_year
    # Define the upper and lower magnitude bounds of the bin
    mlow = row[1]
    mhigh = row[1] + bin_width
    midpoints[i] = (mlow + mhigh) / 2.
    # From the selected years find which earthquakes are within the magnitude range
    selected_magnitudes = np.logical_and(
        magnitudes[selected_years] >= mlow,
        magnitudes[selected_years] < mhigh
    )
    # Count the number of earthquakes selected
    n_mags[i] = np.sum(selected_magnitudes)
    # Divide by the duration of completeness for the magnitude bin
    rates[i] = float(n_mags[i]) / duration[i]

    
# Let's take a look at what we've just counted
print("Mlow   Mmid   Mhigh   Comp. Year  Num.Events   Rate (/yr)")
for i in range(nbins):
    print("%.1f     %.1f   %.2f          %.0f         %3g   %.6f" % (
        completeness_table[i, 1],
        completeness_table[i, 1] + bin_width,
        completeness_table[i, 1] + (bin_width / 2.),
        completeness_table[i, 0],
        n_mags[i],
        rates[i]
    ))

we can also plot a figure

In [ ]:
plt.figure(figsize=(10,10))
plt.bar(completeness_table[:, 1], rates, 0.5, log=True)
plt.grid()
plt.xlabel("Magnitude", fontsize=14)
plt.ylabel("Normalised Rate /yr", fontsize=14)

We can also count the cumulative rates greater than or equal to each magnitude bin

In [ ]:
# Get the cumulative rates
cum_rates = np.zeros_like(n_mags)
for i in range(nbins):
    # Sum the rates greater than or equal to each bin
    cum_rates[i] = np.sum(rates[i:])
    
# Add this information onto the table
print("Mlow   Mmid   Mhigh   Comp. Year  Num.Events   Rate (/yr)   Cum. Rate")
for i in range(nbins):
    print("%.1f     %.1f   %.2f          %.0f         %3g   %8.6f   %8.6f" %(
        completeness_table[i, 1],
        completeness_table[i, 1] + bin_width,
        completeness_table[i, 1] + (bin_width / 2.),
        completeness_table[i, 0],
        n_mags[i],
        rates[i],
        cum_rates[i]
    ))

In [ ]:
plt.figure(figsize=(10,10))
plt.bar(completeness_table[:, 1], rates, 0.5, log=True)
plt.plot(midpoints, rates, "ko-", lw=2, label="incremental")
plt.plot(midpoints, cum_rates, "rs-", lw=2, label="cumulative")
plt.legend()
plt.grid()
plt.xlabel("Magnitude", fontsize=14)
plt.ylabel("Normalised Rate /yr", fontsize=14)

### The Gutenberg-Richter Model

Looking at the data we can see that once we account for completeness in the catalogue, the relation between magnitude and the common logarithm of the <i>cumulative</i> number of events per year greater than that magnitude is (approximately) linear:

$\log_{10} \left( {Nc} \right) = a - bM$

Can we fit the a- and b-value?

Well let's start by looking at the data in the common logarithmic space

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(midpoints, np.log10(rates), "ko-", lw=2, label="incremental")
plt.plot(midpoints, np.log10(cum_rates), "rs-", lw=2, label="cumulative")
plt.xlabel("Magnitude", fontsize=14)
plt.ylabel(r"$\log_{10} \left( {N_C} \right)$", fontsize=14)
plt.legend()
plt.grid()

Looks like a straight line fit could work!

Let's do simple linear regression using a tool from numpy

In [ ]:
# Produces two outputs - the best fitting parameters and the covariance matrix
best_fit, C = np.polyfit(midpoints, # x-values, use the midpoints of the magnitude bin
                         np.log10(cum_rates),  # y-values, use the log10 of the cumulative rates
                         deg=1, # order of polynomial - in this case linear (order = 1)
                         cov=True)  # Also return the covariance matrix

# The variance of the best fitting parameters can be found on the leading diagonal of the
# covarince matrix. Take the square root of these to find the standard deviations
uncertainties = np.sqrt(np.diag(C))
# Best-fit is an array with the [b-value, a-value]
bls, als = best_fit
# b-value is negative in this case, so turn it positive
bls = np.fabs(bls)
print("a = %.4f (+/- %.4f)  b = %.4f (+/- %.4f)" %
      (als, uncertainties[1], bls, uncertainties[0]))

Great! We have our Gutenberg-Richter model, right? $\log_{10} \left( {Nc} \right) = a - bM$

Well, yes. But ask yourself the following ...

1. How confident are we that the rates we observe in each bin are representative of the long term rates for that bin? How many earthquakes do we have in each bin?

2. If we are fitting to the cumulative rates, is the rate of events with M > 5 an observation independent of those rates with M > 6?

### The Weichert (1980) Approach

<b>Don't worry too much about the mathematics, this is just here for completeness!</b>

Weichert (1980) proposes a maximum likelihood approach for finding parameters of the Gutenberg-Richter model for binned magnitudes of time-varying completeness. See the paper for the full derivation of the model.

Firstly, there is some calculus involved here, so we switch from the common logarithm to the natural logarithm.

$N_C = 10^{a - b \cdot M} = e^{\alpha - \beta \cdot M}$

where $\alpha = a\cdot\ln\left( {10} \right)$ and $\beta = b\cdot\ln\left( {10} \right)$

The likelihood function for the estimation of $\beta$ is found to be:

$L \left( {\beta | n_i, m_i, t_i} \right) = \frac{N!}{\prod_i n_i !}\cdot\prod_i \left( {\frac{t_i \exp\left( {-\beta m_i} \right)}{\sum_j t_j \exp \left( {-\beta m_j} \right)}} \right) ^{n_i}$

where $n_i$ is the number of earthquakes in magnitude bin $m_i$, with the completeness duration $t_i$, and $N$ is the total number of earthquakes <i>within the period of completeness</i>.


This has a maximum at $\ln L$ when:

$\frac{\sum_i t_i m_i \exp \left( {-\beta m_i} \right)}{\sum_j t_j \exp \left( {-\beta m_j} \right)} - \frac{\sum n_i m_i}{N} = 0$


The value for $\beta$ can be solved by iteration or Newton conjugate-gradient method (as below). Once this is known the rate of earthquakes above the minimum magnitude $m_{min}$ is determined from:

$N_C \left( {m \geq m_{min}} \right) = N \cdot \frac{\sum_i \exp\left( {-\beta m_i} \right)}{\sum_j t_j \exp\left( {-\beta m_j} \right)}$

From this we can infer a-value from the original form of the G-R model:

$a = \log_{10} \left( {N_C \left[{m \geq m_{min}}\right]} \right) + b \cdot m_{min}$


In [ ]:
# This defines our likelihood function
def likelihood(beta, mags, nmags, duration):
    """
    Returns the minimum of the log-likelihood value. An optimum bvalue should bring
    this close to 0.0
    :param bvalue:
        b-value
    :param mags:
        mid-points of the magnitude bins
    :param nmags:
        Number of events in the specific magnitude bins
    :param duration:
        Completeness durations (in years) of the magnitude bins
    """
    mbar = np.sum(nmags * mags) / float(np.sum(nmags))
    numerator = np.sum(duration * mags * np.exp(-beta * mags))
    denominator = np.sum(duration * np.exp(-beta * mags))
    value = (numerator / denominator) - mbar
    return np.fabs(value)

# This applies the Weichert method of maximum likelihood estimation
def weichert(mags, nmags, duration, b0=1.0):
    """
    Implements the Weichert maximum likelihood estimator of rate and
    b-value for catalogues with time-varying completeness
    :param mags:
        mid-points of the magnitude bins
    :param nmags:
        Number of events in the specific magnitude bins
    :param duration:
        Completeness durations (in years) of the magnitude bins
    :param b0:
        Initial guess of b-value (optional, default = 1)
    :returns:
        a-value, b-value, stddev_a, stddev_b, rate_m0, stddev_rate_m0
    """
    # Initial guess at b-value, turn to beta
    beta0 = b0 * np.log(10.)

    # Find the optimum b-value such that the loglikelihood is equal to 0
    # For this we use scipy's fsolve function
    [beta] = fsolve(likelihood, x0=beta0, args=(mags, nmags, duration))
    b = beta / np.log(10.)

    # Now we have the b-value, need to get a-value
    # Get total number of earthquakes
    neq = float(np.sum(nmags))
    
    # We see this exp(-beta * m) being used a lot, so just calculate once and
    # save as a variable
    e_beta_m = np.exp(-beta * mags)
    
    # Get the rate of events above Mmin
    rate_m0 = neq * np.sum(e_beta_m) / np.sum(duration * e_beta_m)

    # Get the a-value from G-R: a = Nc + b * mmin
    a = np.log10(rate_m0) + b * mags[0]

    # Get the variance of b (from equation 9 of Weichert, 1980)
    var_beta = ((np.sum(duration * e_beta_m)) ** 2.) /\
        ((np.sum(duration * mags * e_beta_m) ** 2.) -
         (np.sum(duration * e_beta_m) * np.sum(duration * (mags ** 2.) * e_beta_m)))
    var_beta = (-1.0 / neq) * var_beta
    stddev_b = np.sqrt(var_beta) / np.log(10.)

    # Get the variance of rate >= M0
    stddev_rate_m0 = np.sqrt(rate_m0 / neq)
    return a, b, stddev_b, rate_m0, stddev_rate_m0


Now to run the Weichert function

In [ ]:
awc, bwc, sigma_bwc, rate_m0, stddev_rate_m0 = weichert(midpoints,
                                                        n_mags.astype(float),
                                                        duration)
print("a = %.4f  b = %.4f (+/- %.4f)  Rate M >= %.2f is %.3f (+/- %.4f))" % 
      (awc, bwc, sigma_bwc, completeness_table[0, 1], rate_m0, stddev_rate_m0))

Now we have two different estimates of a- and b-value and they seem to agree with each other!

So, let's take a look at them

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(midpoints, np.log10(cum_rates), "o", label="observation")
plt.plot(midpoints, awc - bwc * midpoints, "k-", lw=2, label="Weichert")
plt.plot(midpoints, als - bls * midpoints, "r-", lw=2, label="Linear LSQ.")
plt.grid()
plt.legend()
plt.xlabel("M", fontsize=14)
plt.ylabel(r"$\log_{10} N_C$", fontsize=14)

# Can we now answer the questions?

Using the a- and b-values we found (don't worry about uncertainties right now) ...

How often should we expect an earthquake of magnitude greater than or equal to 6.0?

In [ ]:
rate_M6above = 10.0 ** (awc - bwc * 6.0)
print("The rate of events above magnitude 6 is %.3f per year" % rate_M6above)

# The end of the story?

Not quite ...

Theoretically the Gutenberg-Richter model is unbounded, but we have seen in the lectures that is not really the case. In practice we use a form of the model that places an upper bound ($M_{MAX}$) and a lower bound ($M_{MIN}$) on the relation.

Again, we will skip the derivation, but the final model is:

$\lambda_m = \nu \frac{\exp\left[ {-\beta\left( {m - m_{min}}\right)}\right] - \exp\left[ {-\beta\left( {m_{max} - m_{min}}\right)}\right]}{1 - \exp\left[ {-\beta\left( {m_{max} - m_{min}}\right)}\right]}$

where $\nu$ is the rate of events above $M_{min}$ and $\beta = b\cdot \ln\left( {10} \right)$


In [ ]:
# Code this into a small function
def bounded_gutenberg_richter(m, a, b, mmin, mmax):
    rate_m0 = 10.0 ** (a - b * mmin)
    beta = b * np.log(10.)
    rate = rate_m0 * (np.exp(-beta * (m - mmin)) - np.exp(-beta * (mmax - mmin))) /\
        (1.0 - np.exp(-beta * (mmax - mmin)))
    return rate

### Maximum magnitude ...

The calculation (estimation, really) of the maximum magnitude is a whole other story!

For the time being, and for your projects, you could:

1. Just use the largest in your catalogue (obs. Mmax)

2. Use the largest in the catalogue plus an additional increment (e.g. obs. Mmax + 0.5, + 1.0 etc.)

3. Find out the largest event to have occurred in your region?

4. Find out the largest event to have occurred in tectonic environments similar to yours?


In [ ]:
# Maximum observed
mmax_obs = np.max(magnitudes)
mmax = mmax_obs + 0.5

In [ ]:
plt.figure(figsize=(8,8))
plt.semilogy(midpoints, cum_rates, "o")
plt.semilogy(midpoints, 10.0 ** (awc - bwc * midpoints), "k-", lw=2, label="Weichert")
plt.semilogy(midpoints, 10.0 ** (als - bls * midpoints), "r-", lw=2, label="Linear LSQ.")
reference_magnitudes = np.arange(3., 7.2, 0.05)
model_cum_rates = bounded_gutenberg_richter(reference_magnitudes,
                                            awc, bwc, 3.0, mmax)
plt.semilogy(reference_magnitudes, model_cum_rates, "b--", lw=2, label="Bounded GR\nfrom Weichert")
plt.legend(loc=3)
plt.grid()
plt.xlabel("M", fontsize=14)
plt.ylabel(r"$N_C$", fontsize=14)

## Finally ... can we answer question 2?

What is the probability of observing this earthquake (e.g. M > m) in a 50 year period?

The Gutenberg-Richter model (bounded or unbounded) can tell us the annual rate of events exceeding a given magnitude. But how can we use this to answer the question of the probability of an event in a given time period, T?

For this we use the Poisson model

$P\left( {M > m | T} \right) = 1.0 - \exp\left( {-\lambda T} \right)$

where $\lambda$ is the annual rate of events with $M > m$ and T is the time period.

So, from the model above, what is the probability of experiencing an earthquake with magnitude greater than or equal to M 6.0 in a 10 year time period?

In [ ]:
# Let's use the bounded Gutenberg-Richter
rate_mgt6 = bounded_gutenberg_richter(6, awc, bwc, 3.0, mmax)
print("The rate of evetns with M >= 6 is %.4f" % rate_mgt6)

prob_10yrs = 1.0 - np.exp(-rate_mgt6 * 10.0)
print("The probability of an event with M >= 6 in 10 years is %.4f" % prob_10yrs)